In [2]:
!pip install opendatasets as od
import opendatasets as od
od.download("https://www.kaggle.com/datasets/senapatirajesh/netflix-tv-shows-and-movies")

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: omaratef3221
Your Kaggle Key: ········


100%|██████████████████████████████████████| 1.17M/1.17M [00:00<00:00, 1.27MB/s]

In [27]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import dotenv

In [12]:
data_df = pd.read_csv("netflix-tv-shows-and-movies/NetFlix.csv")[["title","description"]]
print(data_df.isna().sum())
print()
print(data_df.shape)

title          0
description    0
dtype: int64

(7787, 2)


In [25]:
Embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                            model_kwargs={'device': 'mps'})

In [32]:
REVIEWS_CSV_PATH = "data/reviews.csv"
REVIEWS_CHROMA_PATH = "chroma_data"

dotenv.load_dotenv()

loader = DataFrameLoader(data_df, page_content_column="description")
descriptions = loader.load()

reviews_vector_db = Chroma.from_documents(
    descriptions, Embeddings, persist_directory=REVIEWS_CHROMA_PATH
)

In [137]:
question = """
movie talking about romantic love story between couple fighting to live together
"""

relevant_docs = reviews_vector_db.similarity_search(question, k=3)

In [138]:
print("Most Suggested Movie:")
print()
title = relevant_docs[0].metadata["title"]
print("Movie Name: ", data_df[data_df["title"] == title]["title"].iloc[0])
print()
print("Movie Description: ", data_df[data_df["title"] == title]["description"].iloc[0])

print()
print()

print("Least Suggested Movie:")
print()
title = relevant_docs[-1].metadata["title"]
print("Movie Name: ", data_df[data_df["title"] == title]["title"].iloc[0])
print()
print("Movie Description: ", data_df[data_df["title"] == title]["description"].iloc[0])

Most Suggested Movie:

Movie Name:  All Good Ones Get Away

Movie Description:  When a mysterious figure blackmails an adulterous couple during a romantic getaway, their secret affair turns into a fight for survival.


Least Suggested Movie:

Movie Name:  All About Love

Movie Description:  An inseparable couple struggles to stay together when betrayal threatens to tear their lives apart.


In [ ]:
reviews_vector_db = Chroma(persist_directory=REVIEWS_CHROMA_PATH, embedding_function=Embeddings)